In [7]:
import requests
import re

# For local streaming, the websockets are hosted without ssl - http://
HOST = 'localhost:5000'
URI = f'http://{HOST}/api/v1/generate'

# For reverse-proxied streaming, the remote will likely host with ssl - https://
# URI = 'https://your-uri-here.trycloudflare.com/api/v1/generate'


def run(prompt,test):
    request = {
        'prompt': prompt.replace("{test}",test),
        'max_new_tokens': 250,
        'temperature': 0.0,
        'top_p': 0.1,
        'length_penalty': 5,
        'early_stopping': True,
        'seed': 0,
    }

    response = requests.post(URI, json=request)
    print(response.json()['results'][0]['text'])

    return response.json()['results'][0]['text']


prompt = '''
[INST]
You are a medical expert. You are tasked to identify causality between two conditions.
Return yes if A causes B. If B causes A or if you are not sure, return no. Keep your response short and succinct.
Input
{test}
[/INST]
'''
run(prompt,"A: uti, B: sepsis")

Yes, A (uti) can cause B (sepsis).


'Yes, A (uti) can cause B (sepsis).'

In [8]:
import pandas as pd

In [10]:
symptom=["uti", "low oxygen", "low neutrophils", "low BP", "high BP", "age>50"]
df = pd.DataFrame(symptom, columns=['symptom'])

In [11]:
df['relation'] = df.apply(lambda x: run(prompt, f"A: {x.symptom}, B: sepsis"), axis=1)

Yes, A (uti) can cause B (sepsis).
Yes, A (low oxygen) causes B (sepsis). Low oxygen levels can lead to sepsis, a life-threatening condition that can cause organ failure and death.
Yes, A (low neutrophils) causes B (sepsis). Low neutrophil counts can impair the body's ability to fight off infections, which can lead to the development of sepsis.
No. Sepsis can cause low blood pressure (BP), but low BP does not cause sepsis.
Yes, A (high BP) can cause B (sepsis). High blood pressure can lead to damage to blood vessels, which can increase the risk of bacterial infection and sepsis.
Yes, A (age > 50) causes B (sepsis). As people age, their immune systems weaken, making them more susceptible to infections like sepsis.


In [12]:
df['result'] = df.apply(lambda x: 'Yes' if 'Yes' in x.relation else 'No', axis=1)

In [13]:
df

,symptom,relation,result
0,uti,"Yes, A (uti) can cause B (sepsis).",Yes
1,low oxygen,"Yes, A (low oxygen) causes B (sepsis). Low oxy...",Yes
2,low neutrophils,"Yes, A (low neutrophils) causes B (sepsis). Lo...",Yes
3,low BP,"No. Sepsis can cause low blood pressure (BP), ...",No
4,high BP,"Yes, A (high BP) can cause B (sepsis). High bl...",Yes
5,age>50,"Yes, A (age > 50) causes B (sepsis). As people...",Yes
